In [10]:
import pandas as pd
import MetaTrader5 as mt5
from backtesting import Backtest, Strategy
import logging
from datetime import datetime
import pytz
import matplotlib.pyplot as plt

# Logging configuration
logging.basicConfig(filename='backtest.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# MetaTrader 5 initialization
def init_mt5_connection(login, password, server):
    if not mt5.initialize(login=login, password=password, server=server):
        logging.error(f"initialize() failed, error code = {mt5.last_error()}")
        sys.exit()
    logging.info("Connected to MetaTrader 5")
    print("Connected to MetaTrader 5")

# Fetch historical OHLC data from MetaTrader 5
def fetch_ohlc_data(symbol, timeframe, start_date, end_date):
    data = mt5.copy_rates_range(symbol, timeframe, start_date, end_date)
    if data is None or len(data) == 0:
        logging.error(f"Failed to fetch data for {symbol}")
        return None
    ohlc_data = pd.DataFrame(data)
    ohlc_data['time'] = pd.to_datetime(ohlc_data['time'], unit='s')
    ohlc_data.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'tick_volume': 'Volume'}, inplace=True)
    return ohlc_data[['time', 'Open', 'High', 'Low', 'Close', 'Volume']]  # Include Volume

# Load and align prediction data
def load_and_align_data(ohlc_data, prediction_file):
    try:
        predictions = pd.read_csv(prediction_file, parse_dates=['time'])
        if 'prediction' not in predictions.columns:
            logging.error(f"'prediction' column not found in {prediction_file}")
            return None
    except Exception as e:
        logging.error(f"Error loading prediction file: {e}")
        return None
    # Merge predictions with OHLC data
    ohlc_data = ohlc_data.merge(predictions[['time', 'prediction']], on='time', how='left')
    ohlc_data['prediction'] = ohlc_data['prediction'].fillna(0)  # Fill missing predictions with 0
    ohlc_data['prediction'] = ohlc_data['prediction'].shift(1)  # Shift predictions to next date
    return ohlc_data

# Backtesting strategy for Buy or Sell
class PredictionStrategy(Strategy):
    risk_reward_ratio = (2, 3)  # Default risk-reward ratio
    signal_type = 'Buy'  # Default signal type
    mean_candle_size = 0.0105  # Default mean candle size

    def init(self):
        # Mean candle size now taken from strategy properties set during strategy initialization
        pass

    def next(self):
        entry_price = self.data.Close[-1]
        risk_part, reward_part = self.risk_reward_ratio
        # Buy signal
        if self.data.prediction[-1] == 1 and self.signal_type == 'Buy':
            sl_price = entry_price - self.mean_candle_size * risk_part
            tp_price = entry_price + self.mean_candle_size * reward_part
            self.buy(sl=sl_price, tp=tp_price)
        # Sell signal
        elif self.data.prediction[-1] == 1 and self.signal_type == 'Sell':
            sl_price = entry_price + self.mean_candle_size * risk_part
            tp_price = entry_price - self.mean_candle_size * reward_part
            self.sell(sl=sl_price, tp=tp_price)

# Function to perform backtesting and save stats/plot
def run_backtest(ohlc_data, strategy_class, risk_reward_ratio, pair_name, signal_type, mean_candle_size):
    strategy_class.risk_reward_ratio = risk_reward_ratio
    strategy_class.signal_type = signal_type
    strategy_class.mean_candle_size = mean_candle_size  # Set mean candle size for the strategy
    bt = Backtest(ohlc_data.set_index('time'), strategy_class, cash=10000, commission=.0003,margin=0.02)
    stats = bt.run()
    plt.figure(figsize=(10, 6))
    bt.plot()
    plt.title(f'Backtest for {pair_name} - {signal_type}')
    #plt.savefig(f'backtest_plot_{pair_name}_{signal_type}.png')
    plt.close()
    stats_df = pd.DataFrame([stats])
    #stats_df.to_csv(f'backtest_stats_{pair_name}_{signal_type}.csv', index=False)
    return stats

# Main function to fetch OHLC data, align it with prediction data, and run backtest for each buy/sell
def main():
    config = {
        'login': 51988090,
        'password': '1fMdV52$74EOcw',
        'server': 'ICMarketsEU-Demo',
        'USDCAD': {
            'symbol': 'USDCAD',
            'timeframe': mt5.TIMEFRAME_D1,
            'mean_candle_size': 0.008664131671633709,
            'buy_prediction_file': 'predUSDCAD_D1_3112buy.csv',
            'buy_risk_reward_ratio': (1, 1),
            'sell_risk_reward_ratio': (1, 2),
        }
    }

    init_mt5_connection(config['login'], config['password'], config['server'])
    utc_from = datetime(2023, 5, 9, tzinfo=pytz.utc)
    utc_to = datetime(2024, 10, 7, tzinfo=pytz.utc)

    for pair_name, pair_config in config.items():
        if pair_name in ['login', 'password', 'server']:
            continue
        logging.info(f"Processing {pair_name}...")
        ohlc_data = fetch_ohlc_data(pair_config['symbol'], pair_config['timeframe'], utc_from, utc_to)
        if ohlc_data is None:
            continue

        if pair_config['buy_prediction_file']:
            ohlc_data_with_predictions = load_and_align_data(ohlc_data, pair_config['buy_prediction_file'])
            if ohlc_data_with_predictions is not None:
                stats = run_backtest(ohlc_data_with_predictions, PredictionStrategy, pair_config['buy_risk_reward_ratio'], pair_name, 'Buy', pair_config['mean_candle_size'])
                print(f"Backtest results for {pair_name} - Buy:\n", stats)
        if pair_config.get('sell_prediction_file'):
            ohlc_data_with_predictions = load_and_align_data(ohlc_data, pair_config['sell_prediction_file'])
            if ohlc_data_with_predictions is not None:
                stats = run_backtest(ohlc_data_with_predictions, PredictionStrategy, pair_config['sell_risk_reward_ratio'], pair_name, 'Sell', pair_config['mean_candle_size'])
                print(f"Backtest results for {pair_name} - Sell:\n", stats)

    mt5.shutdown()

if __name__ == "__main__":
    main()


INFO:bokeh.io.state:Session output file 'PredictionStrategy.html' already exists, will be overwritten.


Connected to MetaTrader 5


c:\ProgramData\anaconda3\envs\forex_env\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\ProgramData\anaconda3\envs\forex_env\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\ProgramData\anaconda3\envs\forex_env\Lib\site-packages\backtesting\_plotting.py:456: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')
c:\ProgramData\anaconda3\envs\forex_env\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
c:\ProgramData\anaconda3\envs\forex_env\Lib\site-packages\

Backtest results for USDCAD - Buy:
 Start                     2023-05-09 00:00:00
End                       2024-10-07 00:00:00
Duration                    517 days 00:00:00
Exposure Time [%]                   80.434783
Equity Final [$]                 42696.526047
Equity Peak [$]                  42696.526047
Return [%]                          326.96526
Buy & Hold Return [%]                1.774547
Return (Ann.) [%]                  170.197825
Volatility (Ann.) [%]             9045.323748
Sharpe Ratio                         0.018816
Sortino Ratio                        1.101693
Calmar Ratio                         1.782312
Max. Drawdown [%]                  -95.492726
Avg. Drawdown [%]                  -42.598141
Max. Drawdown Duration      378 days 00:00:00
Avg. Drawdown Duration       85 days 00:00:00
# Trades                                  117
Win Rate [%]                        61.538462
Best Trade [%]                       0.785096
Worst Trade [%]                     -0.74341